# Model A

In [5]:
# Load required packages
include("../src/EUROml.jl")
using Dates, JLD2, .EUROml

In [3]:
# Load data
results, scorers, shootouts = EUROml.load_data(dir="..")
data, team2id, id2team = EUROml.convert_data(results, scorers, shootouts;output=false)

# Build model
model_build = EUROml.build_model_A(maximum(data.id_home_team))

EuroModel(
    embedding = Embedding(247 => 6),    # 1_482 parameters
    core = Chain(
        layer_1 = Dropout(0.1),
        layer_2 = Dense(14 => 8, relu),  # 120 parameters
        layer_3 = Dense(8 => 4, relu),  # 36 parameters
        layer_4 = Dense(4 => 2),        # 10 parameters
    ),
)         # Total: 1_648 parameters,
          #        plus 2 states.

┌ Warning: `replicate` doesn't work for `TaskLocalRNG`. Returning the same `TaskLocalRNG`.
└ @ LuxCore /home/sebastian/.julia/packages/LuxCore/qiHPC/src/LuxCore.jl:15


In [4]:
# Train model
model, stats, tstate = EUROml.train_model(model_build,data;epochs=500,batch_size=1024,name="./model_A")
jldsave("./model_A.jld2";model,stats,tstate)

In [6]:
# Load model
file = load("./model_A.jld2")
model = file["model"]
stats = file["stats"]
tstate = file["tstate"]

# Predict all matches 
EUROml.predict_games(model,team2id;name="model_A",root_dir="..")